In [ ]:
#install and importing library

!pip install qiskit
!pip install pylatexenc
#!pip install --upgrade qiskit
#!pip install qiskit-aer
#!pip install qiskit-ibmq-provider



#from qiskit import IBMQ , Aer, execute
#from qiskit.circuit import QuantumCircuit, Parameter
from qiskit import QuantumCircuit, transpile
from qiskit.providers.basic_provider import BasicProvider



from qiskit import *
from qiskit.circuit import (
    AnnotatedOperation,
    ControlModifier,
    PowerModifier,
    InverseModifier,
    QuantumCircuit
)
from qiskit.circuit.library import SGate
from qiskit.circuit import Parameter



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136817 sha256=71a998b76fe8829d03d78b1f605918f2347112348888756cbcc47aa5de67b29c
  Stored in directory: /root/.cache/pip/wheels/06/3e/78/fa1588c1ae991bbfd814af2bcac6cef7a178beee1939180d46
Successfully built pylatexenc


In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit.providers.basic_provider import BasicSimulator
import numpy as np

# --- Helper Functions ---

def encode_message(bits, bases):
    """Encode Alice's bits into qubits based on chosen bases."""
    circuits = []
    for bit, base in zip(bits, bases):
        qc = QuantumCircuit(1, 1)
        if bit == 1:
            qc.x(0)
        if base == 1:
            qc.h(0)
        circuits.append(qc)
    return circuits

def measure_message(circuits, bases):
    """Measure qubits in Bob's chosen bases."""
    measured_circuits = []
    for qc, base in zip(circuits, bases):
        c = qc.copy()
        if base == 1:
            c.h(0)
        c.measure(0, 0)
        measured_circuits.append(c)
    return measured_circuits

def remove_garbage(alice_bases, bob_bases, bits):
    """Discard bits where Alice and Bob used different bases."""
    return [bit for a_base, b_base, bit in zip(alice_bases, bob_bases, bits) if a_base == b_base]

def calculate_qber(alice_key, bob_key):
    """Calculate Quantum Bit Error Rate (QBER)."""
    if len(alice_key) == 0:
        return 0
    errors = sum(a != b for a, b in zip(alice_key, bob_key))
    return errors / len(alice_key)

# --- BB84 Protocol ---

def bb84_protocol(n_bits=16, eavesdrop=False, seed=0):
    np.random.seed(seed)

    # Step 1: Alice prepares bits & bases
    alice_bits = np.random.randint(2, size=n_bits)
    alice_bases = np.random.randint(2, size=n_bits)
    message = encode_message(alice_bits, alice_bases)

    # Optional: Eve intercepts and measures
    if eavesdrop:
        eve_bases = np.random.randint(2, size=n_bits)
        eve_circuits = measure_message(message, eve_bases)
        backend = BasicSimulator()
        eve_transpiled = transpile(eve_circuits, backend)
        eve_job = backend.run(eve_transpiled, shots=1)
        eve_results = eve_job.result()
        eve_measured_bits = []
        for res in eve_results.results:
            key = list(res.data.counts.keys())[0]
            eve_measured_bits.append(int(key, 16))
        # Eve re-encodes message with her measurement (disturbs qubits)
        message = encode_message(eve_measured_bits, eve_bases)

    # Step 2: Bob chooses bases & measures
    bob_bases = np.random.randint(2, size=n_bits)
    bob_circuits = measure_message(message, bob_bases)

    backend = BasicSimulator()
    transpiled_circuits = transpile(bob_circuits, backend)
    job = backend.run(transpiled_circuits, shots=1)
    results = job.result()

    bob_measured_bits = []
    for res in results.results:
        key = list(res.data.counts.keys())[0]  # "0x0" or "0x1"
        bob_measured_bits.append(int(key, 16))  # convert from hex

    # Step 3: Sifting
    alice_key = remove_garbage(alice_bases, bob_bases, alice_bits)
    bob_key = remove_garbage(alice_bases, bob_bases, bob_measured_bits)

    qber = calculate_qber(alice_key, bob_key)

    return {
        'alice_bits': alice_bits,
        'alice_bases': alice_bases,
        'bob_bases': bob_bases,
        'bob_results': bob_measured_bits,
        'alice_key': alice_key,
        'bob_key': bob_key,
        'qber': qber
    }

# --- Run Example ---

if __name__ == "__main__":
    result_no_eve = bb84_protocol(n_bits=32, eavesdrop=False, seed=42)
    result_with_eve = bb84_protocol(n_bits=32, eavesdrop=True, seed=42)

    print("No Eve:")
    print("Alice key:", result_no_eve['alice_key'])
    print("Bob key:", result_no_eve['bob_key'])
    print("QBER:", result_no_eve['qber'])

    print("\nWith Eve:")
    print("Alice key:", result_with_eve['alice_key'])
    print("Bob key:", result_with_eve['bob_key'])
    print("QBER:", result_with_eve['qber'])


No Eve:
Alice key: [np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(1)]
Bob key: [1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1]
QBER: 0.0

With Eve:
Alice key: [np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(0)]
Bob key: [1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1]
QBER: 0.42857142857142855


In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit.providers.basic_provider import BasicSimulator
import numpy as np

# --- Helper Functions ---

def encode_message(bits, bases):
    circuits = []
    for bit, base in zip(bits, bases):
        qc = QuantumCircuit(1, 1)
        if bit == 1:
            qc.x(0)
        if base == 1:
            qc.h(0)
        circuits.append(qc)
    return circuits

def measure_message(circuits, bases):
    measured_circuits = []
    for qc, base in zip(circuits, bases):
        c = qc.copy()
        if base == 1:
            c.h(0)
        c.measure(0, 0)
        measured_circuits.append(c)
    return measured_circuits

def remove_garbage(alice_bases, bob_bases, bits):
    return [bit for a_base, b_base, bit in zip(alice_bases, bob_bases, bits) if a_base == b_base]

def calculate_qber(alice_key, bob_key):
    if len(alice_key) == 0:
        return 0
    errors = sum(a != b for a, b in zip(alice_key, bob_key))
    return errors / len(alice_key)

# --- BB84 Protocol ---

def bb84_protocol(n_bits=500, eavesdrop=False, seed=0):
    np.random.seed(seed)

    # Alice prepares
    alice_bits = np.random.randint(2, size=n_bits)
    alice_bases = np.random.randint(2, size=n_bits)
    message = encode_message(alice_bits, alice_bases)

    # Eve intercepts (optional)
    if eavesdrop:
        eve_bases = np.random.randint(2, size=n_bits)
        eve_circuits = measure_message(message, eve_bases)
        backend = BasicSimulator()
        eve_transpiled = transpile(eve_circuits, backend)
        eve_job = backend.run(eve_transpiled, shots=1)
        eve_results = eve_job.result()
        eve_measured_bits = []
        for res in eve_results.results:
            key = list(res.data.counts.keys())[0]
            eve_measured_bits.append(int(key, 16))
        message = encode_message(eve_measured_bits, eve_bases)

    # Bob measures
    bob_bases = np.random.randint(2, size=n_bits)
    bob_circuits = measure_message(message, bob_bases)

    backend = BasicSimulator()
    transpiled_circuits = transpile(bob_circuits, backend)
    job = backend.run(transpiled_circuits, shots=1)
    results = job.result()

    bob_measured_bits = []
    for res in results.results:
        key = list(res.data.counts.keys())[0]
        bob_measured_bits.append(int(key, 16))

    alice_key = remove_garbage(alice_bases, bob_bases, alice_bits)
    bob_key = remove_garbage(alice_bases, bob_bases, bob_measured_bits)

    qber = calculate_qber(alice_key, bob_key)

    return {
        'alice_key': alice_key,
        'bob_key': bob_key,
        'qber': qber,
        'key_length': len(alice_key)
    }

# --- Run Example ---

if __name__ == "__main__":
    result_no_eve = bb84_protocol(n_bits=500, eavesdrop=False, seed=42)
    result_with_eve = bb84_protocol(n_bits=500, eavesdrop=True, seed=42)

    print("BB84 with 500 qubits\n")
    print(f"No Eve: key length = {result_no_eve['key_length']}, QBER = {result_no_eve['qber']:.2%}")
    print(f"With Eve: key length = {result_with_eve['key_length']}, QBER = {result_with_eve['qber']:.2%}")


BB84 with 500 qubits

No Eve: key length = 239, QBER = 0.00%
With Eve: key length = 267, QBER = 29.21%


In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit.providers.basic_provider import BasicSimulator
import numpy as np

# --- Helper Functions ---

def encode_message(bits, bases):
    circuits = []
    for bit, base in zip(bits, bases):
        qc = QuantumCircuit(1, 1)
        if bit == 1:
            qc.x(0)
        if base == 1:
            qc.h(0)
        circuits.append(qc)
    return circuits

def measure_message(circuits, bases):
    measured_circuits = []
    for qc, base in zip(circuits, bases):
        c = qc.copy()
        if base == 1:
            c.h(0)
        c.measure(0, 0)
        measured_circuits.append(c)
    return measured_circuits

def remove_garbage(alice_bases, bob_bases, bits):
    return [bit for a_base, b_base, bit in zip(alice_bases, bob_bases, bits) if a_base == b_base]

def calculate_qber(alice_key, bob_key):
    if len(alice_key) == 0:
        return 0
    errors = sum(a != b for a, b in zip(alice_key, bob_key))
    return errors / len(alice_key)

# --- BB84 Protocol with Two Receivers ---

def bb84_two_receivers(n_bits=500, eavesdrop=False, seed=0):
    np.random.seed(seed)

    # Alice prepares
    alice_bits = np.random.randint(2, size=n_bits)
    alice_bases = np.random.randint(2, size=n_bits)
    message = encode_message(alice_bits, alice_bases)

    # Eve intercepts (optional)
    if eavesdrop:
        eve_bases = np.random.randint(2, size=n_bits)
        eve_circuits = measure_message(message, eve_bases)
        backend = BasicSimulator()
        eve_transpiled = transpile(eve_circuits, backend)
        eve_job = backend.run(eve_transpiled, shots=1)
        eve_results = eve_job.result()
        eve_measured_bits = []
        for res in eve_results.results:
            key = list(res.data.counts.keys())[0]
            eve_measured_bits.append(int(key, 16))
        message = encode_message(eve_measured_bits, eve_bases)

    backend = BasicSimulator()

    # --- Bob 1 ---
    bob1_bases = np.random.randint(2, size=n_bits)
    bob1_circuits = measure_message(message, bob1_bases)
    transpiled1 = transpile(bob1_circuits, backend)
    job1 = backend.run(transpiled1, shots=1)
    results1 = job1.result()
    bob1_measured_bits = [int(list(res.data.counts.keys())[0], 16) for res in results1.results]

    # --- Bob 2 ---
    bob2_bases = np.random.randint(2, size=n_bits)
    bob2_circuits = measure_message(message, bob2_bases)
    transpiled2 = transpile(bob2_circuits, backend)
    job2 = backend.run(transpiled2, shots=1)
    results2 = job2.result()
    bob2_measured_bits = [int(list(res.data.counts.keys())[0], 16) for res in results2.results]

    # --- Sifting & QBER ---
    alice_key_1 = remove_garbage(alice_bases, bob1_bases, alice_bits)
    bob1_key = remove_garbage(alice_bases, bob1_bases, bob1_measured_bits)
    qber1 = calculate_qber(alice_key_1, bob1_key)

    alice_key_2 = remove_garbage(alice_bases, bob2_bases, alice_bits)
    bob2_key = remove_garbage(alice_bases, bob2_bases, bob2_measured_bits)
    qber2 = calculate_qber(alice_key_2, bob2_key)

    return {
        'alice_bits': alice_bits,
        'alice_bases': alice_bases,
        'bob1_bases': bob1_bases,
        'bob1_key': bob1_key,
        'bob2_bases': bob2_bases,
        'bob2_key': bob2_key,
        'alice_key_for_bob1': alice_key_1,
        'alice_key_for_bob2': alice_key_2,
        'qber_bob1': qber1,
        'qber_bob2': qber2
    }

# --- Run Example ---

if __name__ == "__main__":
    result_no_eve = bb84_two_receivers(n_bits=500, eavesdrop=False, seed=42)
    result_with_eve = bb84_two_receivers(n_bits=500, eavesdrop=True, seed=42)

    print("BB84 with Two Receivers and 500 qubits\n")
    print(f"No Eve -> Bob1: key len = {len(result_no_eve['alice_key_for_bob1'])}, QBER = {result_no_eve['qber_bob1']:.2%}")
    print(f"No Eve -> Bob2: key len = {len(result_no_eve['alice_key_for_bob2'])}, QBER = {result_no_eve['qber_bob2']:.2%}")
    print()
    print(f"With Eve -> Bob1: key len = {len(result_with_eve['alice_key_for_bob1'])}, QBER = {result_with_eve['qber_bob1']:.2%}")
    print(f"With Eve -> Bob2: key len = {len(result_with_eve['alice_key_for_bob2'])}, QBER = {result_with_eve['qber_bob2']:.2%}")


BB84 with Two Receivers and 500 qubits

No Eve -> Bob1: key len = 239, QBER = 0.00%
No Eve -> Bob2: key len = 267, QBER = 0.00%

With Eve -> Bob1: key len = 267, QBER = 29.21%
With Eve -> Bob2: key len = 260, QBER = 24.23%


In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit.providers.basic_provider import BasicSimulator
import numpy as np

# --- Helper Functions ---

def encode_message(bits, bases):
    circuits = []
    for bit, base in zip(bits, bases):
        qc = QuantumCircuit(1, 1)
        if bit == 1:
            qc.x(0)
        if base == 1:
            qc.h(0)
        circuits.append(qc)
    return circuits

def measure_message(circuits, bases):
    measured_circuits = []
    for qc, base in zip(circuits, bases):
        c = qc.copy()
        if base == 1:
            c.h(0)
        c.measure(0, 0)
        measured_circuits.append(c)
    return measured_circuits

def remove_garbage(alice_bases, bob_bases, bits):
    return [bit for a_base, b_base, bit in zip(alice_bases, bob_bases, bits) if a_base == b_base]

def calculate_qber(alice_key, bob_key):
    if len(alice_key) == 0:
        return 0
    errors = sum(a != b for a, b in zip(alice_key, bob_key))
    return errors / len(alice_key)

# --- BB84 Protocol with Two Receivers ---

def bb84_two_receivers(n_bits=500, eavesdrop=False, seed=0):
    np.random.seed(seed)

    # Alice prepares
    alice_bits = np.random.randint(2, size=n_bits)
    alice_bases = np.random.randint(2, size=n_bits)
    message = encode_message(alice_bits, alice_bases)

    # Eve intercepts (optional)
    eve_measured_bits = []
    eve_bases = []
    if eavesdrop:
        eve_bases = np.random.randint(2, size=n_bits)
        eve_circuits = measure_message(message, eve_bases)
        backend = BasicSimulator()
        eve_transpiled = transpile(eve_circuits, backend)
        eve_job = backend.run(eve_transpiled, shots=1)
        eve_results = eve_job.result()
        for res in eve_results.results:
            key = list(res.data.counts.keys())[0]
            eve_measured_bits.append(int(key, 16))
        message = encode_message(eve_measured_bits, eve_bases)

    backend = BasicSimulator()

    # --- Bob 1 ---
    bob1_bases = np.random.randint(2, size=n_bits)
    bob1_circuits = measure_message(message, bob1_bases)
    transpiled1 = transpile(bob1_circuits, backend)
    job1 = backend.run(transpiled1, shots=1)
    results1 = job1.result()
    bob1_measured_bits = [int(list(res.data.counts.keys())[0], 16) for res in results1.results]

    # --- Bob 2 ---
    bob2_bases = np.random.randint(2, size=n_bits)
    bob2_circuits = measure_message(message, bob2_bases)
    transpiled2 = transpile(bob2_circuits, backend)
    job2 = backend.run(transpiled2, shots=1)
    results2 = job2.result()
    bob2_measured_bits = [int(list(res.data.counts.keys())[0], 16) for res in results2.results]

    # --- Sifting & QBER ---
    alice_key_1 = remove_garbage(alice_bases, bob1_bases, alice_bits)
    bob1_key = remove_garbage(alice_bases, bob1_bases, bob1_measured_bits)
    qber1 = calculate_qber(alice_key_1, bob1_key)

    alice_key_2 = remove_garbage(alice_bases, bob2_bases, alice_bits)
    bob2_key = remove_garbage(alice_bases, bob2_bases, bob2_measured_bits)
    qber2 = calculate_qber(alice_key_2, bob2_key)

    return {
        'alice_bits': alice_bits,
        'alice_bases': alice_bases,
        'bob1_bases': bob1_bases,
        'bob1_key': bob1_key,
        'bob2_bases': bob2_bases,
        'bob2_key': bob2_key,
        'alice_key_for_bob1': alice_key_1,
        'alice_key_for_bob2': alice_key_2,
        'qber_bob1': qber1,
        'qber_bob2': qber2,
        'eve_bases': eve_bases,
        'eve_measured_bits': eve_measured_bits
    }

# --- Run Example ---

if __name__ == "__main__":
    result_no_eve = bb84_two_receivers(n_bits=500, eavesdrop=False, seed=42)
    result_with_eve = bb84_two_receivers(n_bits=500, eavesdrop=True, seed=42)

    print("BB84 with Two Receivers and 500 qubits\n")
    print(f"No Eve -> Bob1: key len = {len(result_no_eve['alice_key_for_bob1'])}, QBER = {result_no_eve['qber_bob1']:.2%}")
    print(f"No Eve -> Bob2: key len = {len(result_no_eve['alice_key_for_bob2'])}, QBER = {result_no_eve['qber_bob2']:.2%}")
    print()
    print(f"With Eve -> Bob1: key len = {len(result_with_eve['alice_key_for_bob1'])}, QBER = {result_with_eve['qber_bob1']:.2%}")
    print(f"With Eve -> Bob2: key len = {len(result_with_eve['alice_key_for_bob2'])}, QBER = {result_with_eve['qber_bob2']:.2%}")

    # --- Eve key analysis ---
    def sift_eve_key(alice_bases, eve_bases, eve_measured_bits):
        return [bit for a, e, bit in zip(alice_bases, eve_bases, eve_measured_bits) if a == e]

    def calculate_similarity(key1, key2):
        if len(key1) == 0 or len(key2) == 0:
            return 0
        length = min(len(key1), len(key2))
        matches = sum(a == b for a, b in zip(key1[:length], key2[:length]))
        return matches / length

    # Eve's sifted key
    eve_key = sift_eve_key(result_with_eve['alice_bases'],
                           result_with_eve['eve_bases'],
                           result_with_eve['eve_measured_bits'])
    print(f"\nEve's sifted key length: {len(eve_key)}")

    # Similarities
    similarity_bob1 = calculate_similarity(eve_key, result_with_eve['bob1_key'])
    similarity_bob2 = calculate_similarity(eve_key, result_with_eve['bob2_key'])
    print(f"Similarity between Eve and Bob1 keys: {similarity_bob1:.2%}")
    print(f"Similarity between Eve and Bob2 keys: {similarity_bob2:.2%}")


BB84 with Two Receivers and 500 qubits

No Eve -> Bob1: key len = 239, QBER = 0.00%
No Eve -> Bob2: key len = 267, QBER = 0.00%

With Eve -> Bob1: key len = 267, QBER = 29.21%
With Eve -> Bob2: key len = 260, QBER = 24.23%

Eve's sifted key length: 239
Similarity between Eve and Bob1 keys: 57.32%
Similarity between Eve and Bob2 keys: 53.14%


Intercept Resend

In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit.providers.basic_provider import BasicSimulator
import numpy as np

# --- Helper Functions ---

def encode_message(bits, bases):
    circuits = []
    for bit, base in zip(bits, bases):
        qc = QuantumCircuit(1, 1)
        if bit == 1:
            qc.x(0)
        if base == 1:
            qc.h(0)
        circuits.append(qc)
    return circuits

def measure_message(circuits, bases):
    measured_circuits = []
    for qc, base in zip(circuits, bases):
        c = qc.copy()
        if base == 1:
            c.h(0)
        c.measure(0, 0)
        measured_circuits.append(c)
    return measured_circuits

def remove_garbage(alice_bases, bob_bases, bits):
    return [bit for a_base, b_base, bit in zip(alice_bases, bob_bases, bits) if a_base == b_base]

def calculate_qber(alice_key, bob_key):
    if len(alice_key) == 0:
        return 0
    errors = sum(a != b for a, b in zip(alice_key, bob_key))
    return errors / len(alice_key)

def calculate_similarity(alice_bits, eve_bits):
    if eve_bits is None:
        return None
    correct = sum(a == e for a, e in zip(alice_bits, eve_bits))
    return correct / len(alice_bits)

# --- BB84 Protocol with Two Receivers and Independent Eve Attacks ---

def bb84_two_receivers(n_bits=500, eavesdrop=False, seed=0):
    np.random.seed(seed)

    # Alice prepares
    alice_bits = np.random.randint(2, size=n_bits)
    alice_bases = np.random.randint(2, size=n_bits)
    message = encode_message(alice_bits, alice_bases)

    backend = BasicSimulator()

    # --- Eve Intercepts Bob1's Channel ---
    eve1_measured_bits = None
    if eavesdrop:
        eve1_bases = np.random.randint(2, size=n_bits)
        eve1_circuits = measure_message(message, eve1_bases)
        eve1_transpiled = transpile(eve1_circuits, backend)
        eve1_job = backend.run(eve1_transpiled, shots=1)
        eve1_results = eve1_job.result()
        eve1_measured_bits = [int(list(res.data.counts.keys())[0], 16) for res in eve1_results.results]
        message_for_bob1 = encode_message(eve1_measured_bits, eve1_bases)
    else:
        message_for_bob1 = message

    # --- Eve Intercepts Bob2's Channel ---
    eve2_measured_bits = None
    if eavesdrop:
        eve2_bases = np.random.randint(2, size=n_bits)
        eve2_circuits = measure_message(message, eve2_bases)
        eve2_transpiled = transpile(eve2_circuits, backend)
        eve2_job = backend.run(eve2_transpiled, shots=1)
        eve2_results = eve2_job.result()
        eve2_measured_bits = [int(list(res.data.counts.keys())[0], 16) for res in eve2_results.results]
        message_for_bob2 = encode_message(eve2_measured_bits, eve2_bases)
    else:
        message_for_bob2 = message

    # --- Bob 1 ---
    bob1_bases = np.random.randint(2, size=n_bits)
    bob1_circuits = measure_message(message_for_bob1, bob1_bases)
    transpiled1 = transpile(bob1_circuits, backend)
    job1 = backend.run(transpiled1, shots=1)
    results1 = job1.result()
    bob1_measured_bits = [int(list(res.data.counts.keys())[0], 16) for res in results1.results]

    # --- Bob 2 ---
    bob2_bases = np.random.randint(2, size=n_bits)
    bob2_circuits = measure_message(message_for_bob2, bob2_bases)
    transpiled2 = transpile(bob2_circuits, backend)
    job2 = backend.run(transpiled2, shots=1)
    results2 = job2.result()
    bob2_measured_bits = [int(list(res.data.counts.keys())[0], 16) for res in results2.results]

    # --- Sifting & QBER ---
    alice_key_1 = remove_garbage(alice_bases, bob1_bases, alice_bits)
    bob1_key = remove_garbage(alice_bases, bob1_bases, bob1_measured_bits)
    qber1 = calculate_qber(alice_key_1, bob1_key)

    alice_key_2 = remove_garbage(alice_bases, bob2_bases, alice_bits)
    bob2_key = remove_garbage(alice_bases, bob2_bases, bob2_measured_bits)
    qber2 = calculate_qber(alice_key_2, bob2_key)

    # --- Eve's Success Rate ---
    eve1_similarity = calculate_similarity(alice_bits, eve1_measured_bits)
    eve2_similarity = calculate_similarity(alice_bits, eve2_measured_bits)

    return {
        'alice_bits': alice_bits,
        'alice_bases': alice_bases,
        'bob1_bases': bob1_bases,
        'bob1_key': bob1_key,
        'bob2_bases': bob2_bases,
        'bob2_key': bob2_key,
        'alice_key_for_bob1': alice_key_1,
        'alice_key_for_bob2': alice_key_2,
        'qber_bob1': qber1,
        'qber_bob2': qber2,
        'eve1_bits': eve1_measured_bits,
        'eve2_bits': eve2_measured_bits,
        'eve1_similarity': eve1_similarity,
        'eve2_similarity': eve2_similarity
    }

# --- Run Example ---

if __name__ == "__main__":
    result_no_eve = bb84_two_receivers(n_bits=500, eavesdrop=False, seed=42)
    result_with_eve = bb84_two_receivers(n_bits=500, eavesdrop=True, seed=42)

    print("BB84 with Two Receivers and 500 qubits\n")
    print(f"No Eve -> Bob1: key len = {len(result_no_eve['alice_key_for_bob1'])}, QBER = {result_no_eve['qber_bob1']:.2%}")
    print(f"No Eve -> Bob2: key len = {len(result_no_eve['alice_key_for_bob2'])}, QBER = {result_no_eve['qber_bob2']:.2%}")
    print()
    print(f"With Eve -> Bob1: key len = {len(result_with_eve['alice_key_for_bob1'])}, QBER = {result_with_eve['qber_bob1']:.2%}")
    print(f"With Eve -> Bob2: key len = {len(result_with_eve['alice_key_for_bob2'])}, QBER = {result_with_eve['qber_bob2']:.2%}")
    print(f"Eve similarity with Alice (Bob1 channel): {result_with_eve['eve1_similarity']:.2%}")
    print(f"Eve similarity with Alice (Bob2 channel): {result_with_eve['eve2_similarity']:.2%}")


BB84 with Two Receivers and 500 qubits

No Eve -> Bob1: key len = 239, QBER = 0.00%
No Eve -> Bob2: key len = 267, QBER = 0.00%

With Eve -> Bob1: key len = 260, QBER = 24.23%
With Eve -> Bob2: key len = 253, QBER = 24.51%
Eve similarity with Alice (Bob1 channel): 74.60%
Eve similarity with Alice (Bob2 channel): 75.40%


In [ ]:
# pns_bb84_qiskit_snippet.py
from qiskit import QuantumCircuit, transpile
from qiskit.providers.basic_provider import BasicSimulator
import numpy as np

# --- Helper functions (BB84-style) ---
def encode_qubit(bit, base):
    qc = QuantumCircuit(1, 1)
    if bit == 1:
        qc.x(0)
    if base == 1:  # Hadamard for X-basis
        qc.h(0)
    return qc

def measure_qubit_circuit(qc, measure_basis):
    c = qc.copy()
    if measure_basis == 1:
        c.h(0)
    c.measure(0, 0)
    return c

def sift_bits(alice_bases, bob_bases, bits):
    return [b for a, b_b, b in zip(alice_bases, bob_bases, bits) if a == b_b]

def qber(a_key, b_key):
    if len(a_key) == 0:
        return 0.0
    errors = sum(x != y for x, y in zip(a_key, b_key))
    return errors / len(a_key)

# --- PNS-specific utilities ---
def sample_photon_number(mu, size):
    # Poisson-distributed photon numbers per pulse
    return np.random.poisson(mu, size)

# --- PNS BB84 simulation (single receiver, simple) ---
def pns_bb84_sim(n_bits=500, mu=0.8, eavesdrop=False, eve_memory=True, seed=0):
    """
    Simple PNS-style BB84 simulation inspired by SPRINT-PNS ideas.
    - mu: average photon number (WCS)
    - eavesdrop: whether Eve attempts PNS on multi-photon pulses
    - eve_memory: if True, Eve stores her stolen photon and measures AFTER basis reveal
                  if False, Eve measures immediately (random basis)
    Returns a dictionary similar in spirit to your previous script.
    """
    np.random.seed(seed)

    # Alice generation
    alice_bits = np.random.randint(2, size=n_bits)
    alice_bases = np.random.randint(2, size=n_bits)  # 0 = Z, 1 = X

    # photon numbers per pulse
    photon_nums = sample_photon_number(mu, n_bits)

    # prepare per-pulse "sent qubit" circuits for Bob (or None if vacuum)
    circuits_for_bob = []
    bob_expected_from_alice = []  # expected bit value that would reach Bob if measured in Alice's basis
    # Eve storage
    eve_stolen_bits = [None] * n_bits
    eve_stolen_bases = [None] * n_bits

    for i in range(n_bits):
        n = photon_nums[i]
        bit = int(alice_bits[i])
        base = int(alice_bases[i])

        if n == 0:
            # vacuum: no circuit sent to Bob
            circuits_for_bob.append(None)
            bob_expected_from_alice.append(None)
            continue

        # if multi-photon and Eve present -> PNS: she can steal one photon
        if eavesdrop and n >= 2:
            # if Eve has a quantum memory: she waits and measures in correct basis later
            if eve_memory:
                # store the true alice bit & basis (idealized stolen photon)
                eve_stolen_bits[i] = bit
                eve_stolen_bases[i] = base
            else:
                # Eve measures immediately in a random basis (interferes, may introduce errors)
                eve_measure_basis = np.random.randint(2)
                # simulate measurement outcome: if measured in same basis -> gets correct bit, else random
                if eve_measure_basis == base:
                    eve_stolen_bits[i] = bit
                else:
                    eve_stolen_bits[i] = np.random.randint(2)
                eve_stolen_bases[i] = eve_measure_basis

            # After stealing one photon, Eve forwards one photon that we model as identical to Alice's original state
            # (this is the simplified PNS behaviour: remaining photon(s) forwarded to Bob intact)
            # so Bob receives a qubit prepared by Alice's original bit & basis
            circuits_for_bob.append(encode_qubit(bit, base))
            bob_expected_from_alice.append(bit)
        else:
            # single-photon case or no eavesdrop: Bob receives Alice's qubit directly
            circuits_for_bob.append(encode_qubit(bit, base))
            bob_expected_from_alice.append(bit)

    # Bob randomly chooses bases and measures (simulate via Qiskit BasicSimulator per-circuit)
    bob_bases = np.random.randint(2, size=n_bits)
    bob_measured_bits = []
    backend = BasicSimulator()

    # build list of circuits to run (only for pulses that actually present circuits)
    measurement_jobs = []
    ran_indices = []
    for i, (qc, bbase) in enumerate(zip(circuits_for_bob, bob_bases)):
        if qc is None:
            # vacuum -> no detection, represent by None
            bob_measured_bits.append(None)
            continue
        meas_qc = measure_qubit_circuit(qc, bbase)
        measurement_jobs.append(meas_qc)
        ran_indices.append(i)

    if measurement_jobs:
        transpiled = transpile(measurement_jobs, backend)
        job = backend.run(transpiled, shots=1)
        res = job.result()
        # results come in order; extract each bit (counts keys are bitstrings)
        for r in res.results:
            bitstr = list(r.data.counts.keys())[0]
            bob_measured_bits.append(int(bitstr, 16))  # same approach as your code

        # bob_measured_bits currently only contains measured outcomes for non-vacuum pulses in order;
        # we need to merge them back into the full list at the correct indices.
        merged = [None] * n_bits
        m_idx = 0
        for i in range(n_bits):
            if circuits_for_bob[i] is None:
                merged[i] = None
            else:
                merged[i] = bob_measured_bits[m_idx]
                m_idx += 1
        bob_measured_bits = merged
    else:
        # no circuits at all (all vacua)
        bob_measured_bits = [None] * n_bits

    # Sifting: keep only indices where Bob had detection and bases match Alice
    alice_sift = []
    bob_sift = []
    for a_b, b_b, a_bit, b_bit in zip(alice_bases, bob_bases, alice_bits, bob_measured_bits):
        if b_bit is None:
            continue  # Bob had no detection
        if a_b == b_b:
            alice_sift.append(int(a_bit))
            bob_sift.append(int(b_bit))

    # QBER on sifted key
    computed_qber = qber(alice_sift, bob_sift)

    # Eve measurement post-processing (if she had memory she measures in correct basis now)
    if eavesdrop and eve_memory:
        # Eve measures stolen photons in Alice's basis (perfect info for stolen pulses by our simple model)
        eve_measured = []
        for i in range(n_bits):
            if eve_stolen_bits[i] is None:
                eve_measured.append(None)
            else:
                # assume perfect memory: she measures in correct basis -> recovers bit
                eve_measured.append(int(eve_stolen_bits[i]))
    elif eavesdrop and not eve_memory:
        eve_measured = eve_stolen_bits[:]  # already set earlier (may be random for wrong-basis)
    else:
        eve_measured = [None] * n_bits

    # Compute simple similarity measure (only where Eve had a recorded bit)
    def similarity_with_alice(alice_bits, eve_bits):
        pairs = [(a, e) for a, e in zip(alice_bits, eve_bits) if e is not None]
        if not pairs:
            return None
        correct = sum(a == e for a, e in pairs)
        return correct / len(pairs)

    eve_similarity = similarity_with_alice(alice_bits, eve_measured)

    return {
        'alice_bits': alice_bits,
        'alice_bases': alice_bases,
        'photon_numbers': photon_nums,
        'bob_bases': bob_bases,
        'bob_measured_bits': bob_measured_bits,
        'alice_sift': alice_sift,
        'bob_sift': bob_sift,
        'qber': computed_qber,
        'eve_measured': eve_measured,
        'eve_similarity': eve_similarity
    }

# --- Example quick-run ---
if __name__ == "__main__":
    res = pns_bb84_sim(n_bits=300, mu=0.9, eavesdrop=True, eve_memory=True, seed=42)
    print(f"Sifted key length = {len(res['alice_sift'])}, QBER = {res['qber']:.2%}")
    if res['eve_similarity'] is not None:
        print(f"Eve similarity (where she stole a photon) = {res['eve_similarity']:.2%}")
    else:
        print("Eve had no stolen photons in this run.")


Sifted key length = 13, QBER = 61.54%
Eve similarity (where she stole a photon) = 100.00%


In [ ]:
# --- PNS BB84 simulation with two Bobs ---
def pns_bb84_two_bobs(n_bits=500, mu=0.8, eavesdrop=False, eve_memory=True, seed=0):
    """
    BB84 with PNS attack simulation for TWO Bobs.
    Each Bob independently measures incoming qubits.
    """
    np.random.seed(seed)

    # Alice generation
    alice_bits = np.random.randint(2, size=n_bits)
    alice_bases = np.random.randint(2, size=n_bits)  # 0 = Z, 1 = X

    # photon numbers per pulse
    photon_nums = sample_photon_number(mu, n_bits)

    # prepare per-pulse "sent qubit" circuits
    circuits_for_bobs = []
    bob_expected_from_alice = []
    eve_stolen_bits = [None] * n_bits
    eve_stolen_bases = [None] * n_bits

    for i in range(n_bits):
        n = photon_nums[i]
        bit = int(alice_bits[i])
        base = int(alice_bases[i])

        if n == 0:
            circuits_for_bobs.append(None)
            bob_expected_from_alice.append(None)
            continue

        if eavesdrop and n >= 2:
            if eve_memory:
                eve_stolen_bits[i] = bit
                eve_stolen_bases[i] = base
            else:
                eve_measure_basis = np.random.randint(2)
                if eve_measure_basis == base:
                    eve_stolen_bits[i] = bit
                else:
                    eve_stolen_bits[i] = np.random.randint(2)
                eve_stolen_bases[i] = eve_measure_basis

        # Bob(s) receive Alice's original qubit
        circuits_for_bobs.append(encode_qubit(bit, base))
        bob_expected_from_alice.append(bit)

    # Bob bases
    bob1_bases = np.random.randint(2, size=n_bits)
    bob2_bases = np.random.randint(2, size=n_bits)

    # Measure for both Bobs
    def measure_for_bob(bob_bases):
        bob_measured_bits = []
        backend = BasicSimulator()
        measurement_jobs = []
        ran_indices = []

        for i, (qc, bbase) in enumerate(zip(circuits_for_bobs, bob_bases)):
            if qc is None:
                bob_measured_bits.append(None)
                continue
            meas_qc = measure_qubit_circuit(qc, bbase)
            measurement_jobs.append(meas_qc)
            ran_indices.append(i)

        if measurement_jobs:
            transpiled = transpile(measurement_jobs, backend)
            job = backend.run(transpiled, shots=1)
            res = job.result()
            measured_only = [int(list(r.data.counts.keys())[0], 16) for r in res.results]
            merged = [None] * n_bits
            m_idx = 0
            for i in range(n_bits):
                if circuits_for_bobs[i] is None:
                    merged[i] = None
                else:
                    merged[i] = measured_only[m_idx]
                    m_idx += 1
            bob_measured_bits = merged
        else:
            bob_measured_bits = [None] * n_bits

        return bob_measured_bits

    bob1_measured_bits = measure_for_bob(bob1_bases)
    bob2_measured_bits = measure_for_bob(bob2_bases)

    # Sifting for both Bobs
    def sift_for_bob(alice_bases, bob_bases, alice_bits, bob_bits):
        alice_sift = []
        bob_sift = []
        for a_b, b_b, a_bit, b_bit in zip(alice_bases, bob_bases, alice_bits, bob_bits):
            if b_bit is None:
                continue
            if a_b == b_b:
                alice_sift.append(int(a_bit))
                bob_sift.append(int(b_bit))
        return alice_sift, bob_sift

    alice_sift1, bob_sift1 = sift_for_bob(alice_bases, bob1_bases, alice_bits, bob1_measured_bits)
    alice_sift2, bob_sift2 = sift_for_bob(alice_bases, bob2_bases, alice_bits, bob2_measured_bits)

    qber1 = qber(alice_sift1, bob_sift1)
    qber2 = qber(alice_sift2, bob_sift2)

    # Eve post-processing
    if eavesdrop and eve_memory:
        eve_measured = [int(bit) if bit is not None else None for bit in eve_stolen_bits]
    elif eavesdrop and not eve_memory:
        eve_measured = eve_stolen_bits[:]
    else:
        eve_measured = [None] * n_bits

    def similarity_with_alice(alice_bits, eve_bits):
        pairs = [(a, e) for a, e in zip(alice_bits, eve_bits) if e is not None]
        if not pairs:
            return None
        correct = sum(a == e for a, e in pairs)
        return correct / len(pairs)

    eve_similarity = similarity_with_alice(alice_bits, eve_measured)

    return {
        'alice_bits': alice_bits,
        'alice_bases': alice_bases,
        'photon_numbers': photon_nums,
        'bob1_bases': bob1_bases,
        'bob2_bases': bob2_bases,
        'bob1_measured_bits': bob1_measured_bits,
        'bob2_measured_bits': bob2_measured_bits,
        'alice_sift1': alice_sift1,
        'bob_sift1': bob_sift1,
        'qber1': qber1,
        'alice_sift2': alice_sift2,
        'bob_sift2': bob_sift2,
        'qber2': qber2,
        'eve_measured': eve_measured,
        'eve_similarity': eve_similarity
    }

# --- Example quick-run with two Bobs ---
if __name__ == "__main__":
    res = pns_bb84_two_bobs(n_bits=300, mu=0.9, eavesdrop=True, eve_memory=True, seed=42)
    print(f"Bob1 sifted key length = {len(res['alice_sift1'])}, QBER = {res['qber1']:.2%}")
    print(f"Bob2 sifted key length = {len(res['alice_sift2'])}, QBER = {res['qber2']:.2%}")
    if res['eve_similarity'] is not None:
        print(f"Eve similarity (where she stole a photon) = {res['eve_similarity']:.2%}")
    else:
        print("Eve had no stolen photons in this run.")


Bob1 sifted key length = 84, QBER = 0.00%
Bob2 sifted key length = 82, QBER = 0.00%
Eve similarity (where she stole a photon) = 100.00%


PNS

In [ ]:
# --- PNS BB84 simulation with two Bobs and realistic QBER ---
def pns_bb84_two_bobs_realistic(n_bits=500, mu=0.8, eavesdrop=True, eve_memory=True,
                                p_error=0.01, seed=0):
    """
    BB84 with PNS attack simulation for TWO Bobs with realistic measurement errors.
    - p_error: probability of detector error (bit flip)
    """
    np.random.seed(seed)

    # Alice generation
    alice_bits = np.random.randint(2, size=n_bits)
    alice_bases = np.random.randint(2, size=n_bits)  # 0 = Z, 1 = X

    # photon numbers per pulse
    photon_nums = sample_photon_number(mu, n_bits)

    # prepare per-pulse "sent qubit" circuits
    circuits_for_bobs = []
    eve_stolen_bits = [None] * n_bits
    eve_stolen_bases = [None] * n_bits

    for i in range(n_bits):
        n = photon_nums[i]
        bit = int(alice_bits[i])
        base = int(alice_bases[i])

        if n == 0:
            circuits_for_bobs.append(None)
            continue

        # Eve's PNS attack
        if eavesdrop and n >= 2:
            if eve_memory:
                # store for later measurement
                eve_stolen_bits[i] = bit
                eve_stolen_bases[i] = base
            else:
                # immediate measurement in random basis (disturbs state if wrong)
                eve_measure_basis = np.random.randint(2)
                if eve_measure_basis == base:
                    eve_stolen_bits[i] = bit
                else:
                    eve_stolen_bits[i] = np.random.randint(2)
                    # disturbance: flip one photon sent to Bob with 50% chance
                    if np.random.rand() < 0.5:
                        bit = 1 - bit
                eve_stolen_bases[i] = eve_measure_basis

        circuits_for_bobs.append(encode_qubit(bit, base))

    # Bob bases
    bob1_bases = np.random.randint(2, size=n_bits)
    bob2_bases = np.random.randint(2, size=n_bits)

    # Function to measure qubits with realistic errors
    def measure_with_error(qc, measure_basis):
        if qc is None:
            return None
        meas_qc = measure_qubit_circuit(qc, measure_basis)
        backend = BasicSimulator()
        transpiled = transpile(meas_qc, backend)
        job = backend.run(transpiled, shots=1)
        res = job.result()
        bitstr = list(res.get_counts().keys())[0]
        bit = int(bitstr, 2)
        # detector error: flip with probability p_error
        if np.random.rand() < p_error:
            bit = 1 - bit
        return bit

    # Measure for both Bobs
    bob1_measured_bits = [measure_with_error(qc, b) for qc, b in zip(circuits_for_bobs, bob1_bases)]
    bob2_measured_bits = [measure_with_error(qc, b) for qc, b in zip(circuits_for_bobs, bob2_bases)]

    # Sifting function
    def sift_for_bob(alice_bases, bob_bases, alice_bits, bob_bits):
        alice_sift = []
        bob_sift = []
        for a_b, b_b, a_bit, b_bit in zip(alice_bases, bob_bases, alice_bits, bob_bits):
            if b_bit is None:
                continue
            if a_b == b_b:
                alice_sift.append(int(a_bit))
                bob_sift.append(int(b_bit))
        return alice_sift, bob_sift

    alice_sift1, bob_sift1 = sift_for_bob(alice_bases, bob1_bases, alice_bits, bob1_measured_bits)
    alice_sift2, bob_sift2 = sift_for_bob(alice_bases, bob2_bases, alice_bits, bob2_measured_bits)

    qber1 = qber(alice_sift1, bob_sift1)
    qber2 = qber(alice_sift2, bob_sift2)

    # Eve post-processing (if she has memory)
    if eavesdrop and eve_memory:
        eve_measured = [int(bit) if bit is not None else None for bit in eve_stolen_bits]
    elif eavesdrop and not eve_memory:
        eve_measured = eve_stolen_bits[:]
    else:
        eve_measured = [None] * n_bits

    def similarity_with_alice(alice_bits, eve_bits):
        pairs = [(a, e) for a, e in zip(alice_bits, eve_bits) if e is not None]
        if not pairs:
            return None
        correct = sum(a == e for a, e in pairs)
        return correct / len(pairs)

    eve_similarity = similarity_with_alice(alice_bits, eve_measured)

    return {
        'alice_bits': alice_bits,
        'alice_bases': alice_bases,
        'photon_numbers': photon_nums,
        'bob1_bases': bob1_bases,
        'bob2_bases': bob2_bases,
        'bob1_measured_bits': bob1_measured_bits,
        'bob2_measured_bits': bob2_measured_bits,
        'alice_sift1': alice_sift1,
        'bob_sift1': bob_sift1,
        'qber1': qber1,
        'alice_sift2': alice_sift2,
        'bob_sift2': bob_sift2,
        'qber2': qber2,
        'eve_measured': eve_measured,
        'eve_similarity': eve_similarity
    }

# --- Example quick-run ---
if __name__ == "__main__":
    res = pns_bb84_two_bobs_realistic(n_bits=300, mu=0.9, eavesdrop=True, eve_memory=True,
                                      p_error=0.02, seed=42)
    print(f"Bob1 sifted key length = {len(res['alice_sift1'])}, QBER = {res['qber1']:.2%}")
    print(f"Bob2 sifted key length = {len(res['alice_sift2'])}, QBER = {res['qber2']:.2%}")
    if res['eve_similarity'] is not None:
        print(f"Eve similarity (where she stole a photon) = {res['eve_similarity']:.2%}")
    else:
        print("Eve had no stolen photons in this run.")


Bob1 sifted key length = 84, QBER = 1.19%
Bob2 sifted key length = 82, QBER = 2.44%
Eve similarity (where she stole a photon) = 100.00%
